In [1]:
import re
from urllib.request import urlopen

import pandas as pd
import numpy as np


SPREADSHEET_ID = "1YgPc11dgdBUC8jXNp01b7gI6jNHoBRQGwrY_V6lXMgQ"
GSHEET_URL = f"https://docs.google.com/spreadsheet/ccc?key={SPREADSHEET_ID}&output=xlsx"
GSHEET_URL

'https://docs.google.com/spreadsheet/ccc?key=1YgPc11dgdBUC8jXNp01b7gI6jNHoBRQGwrY_V6lXMgQ&output=xlsx'

# Read excel to Dataframe and remove unnecessary columns

In [2]:
# df = pd.read_excel(GSHEET_URL, sheet_name=1)

pal_dict_df = pd.read_excel(
    f"{SPREADSHEET_ID}.xlsx", sheet_name=5, index_col=0, skiprows=1
)
pal_dict_df.drop(["Unnamed: 1", "Unnamed: 3"], axis=1, inplace=True)
pal_dict_df

,Name,Type 1,Type 2,🌜,🔥Kindling,💦Watering,🌱Planting,⚡Electric,🤚Handiwork,🍃Gathering,...,🏹Shot,🛡️Defence,💰Price,🪫Stamina,🚶Walking,🏃Running,🏇Mounted,📦Transporting.1,Capture Multi,Male %
#,,,,,,,,,,,,,,,,,,,,,
1,Lamball,Normal,NaN,NaN,0,0,0,0,1,0,...,70,70,1000,100,40,400,550,160.0,1.5,50
2,Cattiva,Normal,NaN,NaN,0,0,0,0,1,1,...,70,70,1000,100,60,400,550,160.0,1.5,50
3,Chikipi,Normal,NaN,NaN,0,0,0,0,0,1,...,60,60,1000,100,50,375,550,NaN,1.5,50
4,Lifmunk,Leaf,NaN,NaN,0,0,1,0,1,1,...,70,70,1010,100,40,400,550,NaN,0.9,50
5,Foxparks,Fire,NaN,NaN,1,0,0,0,0,0,...,75,70,1040,100,80,400,550,NaN,1.1,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,Paladius,Normal,NaN,NaN,0,0,0,0,0,0,...,120,145,8810,100,100,800,1400,NaN,1.0,50
109,Necromus,Dark,NaN,🌜,0,0,0,0,0,0,...,145,120,8930,100,350,900,1600,NaN,1.0,50
110,Frostallion,Ice,NaN,NaN,0,0,0,0,0,0,...,140,120,8440,300,120,1000,1500,NaN,1.0,50


# Get Emojis unicode range to rename columns of the sheets

In [3]:
emoji_seq_txt = urlopen(
    "https://www.unicode.org/Public/emoji/latest/emoji-sequences.txt"
)
RE_EMOJI_CODES = re.compile(r"^[0-9][A-Z0-9]{3}.{10}; Basic_Emoji.+")

emoji_seq = []
for line in emoji_seq_txt:
    line_txt = line.decode()
    if not re.match(RE_EMOJI_CODES, line_txt):
        continue
    line_txt = line_txt.split(";")[0].replace("FE0F", "").strip()
    if ".." in line_txt:
        emoji_seq += line_txt.split("..")
    else:
        emoji_seq.append(line_txt)

emoji_seq = sorted(set(emoji_seq), key=lambda x: (len(x), x))
emoji_seq[0], emoji_seq[-1]

('00A9', '1FAF8')

# Remove emojis from the columns titles and add prefixes to specific columns

In [4]:
RE_EMOJI = re.compile(r"[\u00A9-\U0001FAF8]")
col_remap = (
    {col: col.lower().replace(" ", "_") for col in pal_dict_df.columns[:3]}
    | {
        col: "work_" + re.sub(RE_EMOJI, "", col.lower().strip())
        for col in pal_dict_df.columns[4:16]
    }
    | {
        col: re.sub(RE_EMOJI, "", col.lower().strip())
        for col in pal_dict_df.columns[16:18]
    }
    | {
        col: "attack_" + re.sub(RE_EMOJI, "", col.lower().strip())
        for col in pal_dict_df.columns[18:22]
    }
    | {
        col: re.sub(RE_EMOJI, "", f"{col.lower().strip()}")
        for col in pal_dict_df.columns[22:23]
    }
    | {
        col: "speed_" + re.sub(RE_EMOJI, "", col.lower().strip()).rstrip(".1")
        for col in pal_dict_df.columns[23:28]
    }
    | {col: col.lower().replace(" ", "_") for col in pal_dict_df.columns[-2:]}
    | {"Male %": "male_percentage", "🌜": "is_nocturnal"}
)
col_remap

{'Name': 'name',
 'Type 1': 'type_1',
 'Type 2': 'type_2',
 '🔥Kindling': 'work_kindling',
 '💦Watering': 'work_watering',
 '🌱Planting': 'work_planting',
 '⚡Electric': 'work_electric',
 '🤚Handiwork': 'work_handiwork',
 '🍃Gathering': 'work_gathering',
 '🪵Lumbering': 'work_lumbering',
 '⛏️Mining': 'work_mining',
 '🥣Medicine': 'work_medicine',
 '❄️Cooling': 'work_cooling',
 '📦Transporting': 'work_transporting',
 '🧑\u200d🌾Farming': 'work_farming',
 '🍗Food': 'food',
 '🥚BreedPWR': 'breedpwr',
 '❤️HP': 'attack_hp',
 '⚔️Melee': 'attack_melee',
 '🏹Shot ': 'attack_shot',
 '🛡️Defence': 'attack_defence',
 '💰Price': 'price',
 '🪫Stamina': 'speed_stamina',
 '🚶Walking': 'speed_walking',
 '🏃Running': 'speed_running',
 '🏇Mounted': 'speed_mounted',
 '📦Transporting.1': 'speed_transporting',
 'Capture Multi': 'capture_multi',
 'Male %': 'male_percentage',
 '🌜': 'is_nocturnal'}

# Apply the columns map remap

In [5]:
pal_dict_df.rename(columns=col_remap, inplace=True)
pal_dict_df

,name,type_1,type_2,is_nocturnal,work_kindling,work_watering,work_planting,work_electric,work_handiwork,work_gathering,...,attack_shot,attack_defence,price,speed_stamina,speed_walking,speed_running,speed_mounted,speed_transporting,capture_multi,male_percentage
#,,,,,,,,,,,,,,,,,,,,,
1,Lamball,Normal,NaN,NaN,0,0,0,0,1,0,...,70,70,1000,100,40,400,550,160.0,1.5,50
2,Cattiva,Normal,NaN,NaN,0,0,0,0,1,1,...,70,70,1000,100,60,400,550,160.0,1.5,50
3,Chikipi,Normal,NaN,NaN,0,0,0,0,0,1,...,60,60,1000,100,50,375,550,NaN,1.5,50
4,Lifmunk,Leaf,NaN,NaN,0,0,1,0,1,1,...,70,70,1010,100,40,400,550,NaN,0.9,50
5,Foxparks,Fire,NaN,NaN,1,0,0,0,0,0,...,75,70,1040,100,80,400,550,NaN,1.1,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,Paladius,Normal,NaN,NaN,0,0,0,0,0,0,...,120,145,8810,100,100,800,1400,NaN,1.0,50
109,Necromus,Dark,NaN,🌜,0,0,0,0,0,0,...,145,120,8930,100,350,900,1600,NaN,1.0,50
110,Frostallion,Ice,NaN,NaN,0,0,0,0,0,0,...,140,120,8440,300,120,1000,1500,NaN,1.0,50


# List out df indices and duplicated indices found

In [6]:
list(pal_dict_df.index)

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 12,
 13,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 31,
 32,
 32,
 33,
 33,
 34,
 35,
 36,
 37,
 37,
 38,
 39,
 40,
 40,
 41,
 42,
 43,
 44,
 45,
 45,
 46,
 47,
 48,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 64,
 65,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 80,
 81,
 81,
 82,
 83,
 84,
 84,
 85,
 85,
 86,
 86,
 87,
 88,
 88,
 89,
 89,
 90,
 90,
 91,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 101,
 102,
 102,
 103,
 104,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 110,
 111]

# Create a new list of indices to remap

In [7]:
new_indices = []
for idx, df_idx in enumerate(pal_dict_df.index, start=1):
    if str(df_idx) in new_indices[:idx]:
        new_indices.append(f"{df_idx}B")
    else:
        new_indices.append(str(df_idx))
new_indices

['1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '12B',
 '13',
 '13B',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '24B',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '31B',
 '32',
 '32B',
 '33',
 '33B',
 '34',
 '35',
 '36',
 '37',
 '37B',
 '38',
 '39',
 '40',
 '40B',
 '41',
 '42',
 '43',
 '44',
 '45',
 '45B',
 '46',
 '47',
 '48',
 '48B',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '58B',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '64B',
 '65',
 '65B',
 '66',
 '67',
 '68',
 '69',
 '70',
 '71',
 '71B',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77',
 '78',
 '79',
 '80',
 '80B',
 '81',
 '81B',
 '82',
 '83',
 '84',
 '84B',
 '85',
 '85B',
 '86',
 '86B',
 '87',
 '88',
 '88B',
 '89',
 '89B',
 '90',
 '90B',
 '91',
 '91B',
 '92',
 '93',
 '94',
 '95',
 '96',
 '97',
 '98',
 '99',
 '100',
 '101',
 '101B',
 '102',
 '102B',
 '103',
 '104',
 '104B',
 '105',
 '106',
 '107',
 '108',
 '109',
 '110',
 '110B',
 '111']

# Apply the indices remap

In [8]:
pal_dict_df.index = new_indices
pal_dict_df[10:20]

,name,type_1,type_2,is_nocturnal,work_kindling,work_watering,work_planting,work_electric,work_handiwork,work_gathering,...,attack_shot,attack_defence,price,speed_stamina,speed_walking,speed_running,speed_mounted,speed_transporting,capture_multi,male_percentage
11,Penking,Water,Ice,NaN,0,2,0,0,2,0,...,95,95,5410,100,110,450,600,280.0,1.0,50
12,Jolthog,Electricity,NaN,NaN,0,0,0,1,0,0,...,75,70,1060,100,60,400,550,NaN,1.2,50
12B,Jolthog Cryst,Ice,NaN,NaN,0,0,0,0,0,0,...,75,80,1070,100,60,400,550,NaN,1.0,50
13,Gumoss,Leaf,Earth,NaN,0,0,1,0,0,0,...,70,70,1310,100,50,300,400,NaN,1.3,50
13B,Gumoss (Special),Leaf,Earth,NaN,0,0,1,0,0,0,...,70,70,1310,100,50,300,400,NaN,1.3,50
14,Vixy,Normal,NaN,NaN,0,0,0,0,0,1,...,70,70,1000,100,60,350,450,NaN,1.3,50
15,Hoocrates,Dark,NaN,🌜,0,0,0,0,0,1,...,70,80,1050,100,70,380,550,NaN,1.0,50
16,Teafant,Water,NaN,NaN,0,1,0,0,0,0,...,60,70,1000,100,60,300,400,NaN,1.3,50
17,Depresso,Dark,NaN,🌜,0,0,0,0,1,0,...,70,70,1050,100,10,300,400,100.0,1.0,50
18,Cremis,Normal,NaN,NaN,0,0,0,0,0,1,...,70,75,1420,100,40,300,450,NaN,1.3,50


# Replace values in `is_nocturnal` from emojis to boolean

In [9]:
pal_dict_df.replace({"is_nocturnal": "🌜"}, True, inplace=True)
pal_dict_df["is_nocturnal"] = pal_dict_df["is_nocturnal"].fillna(False)
pal_dict_df

/tmp/ipykernel_11654/1405662451.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pal_dict_df["is_nocturnal"] = pal_dict_df["is_nocturnal"].fillna(False)


,name,type_1,type_2,is_nocturnal,work_kindling,work_watering,work_planting,work_electric,work_handiwork,work_gathering,...,attack_shot,attack_defence,price,speed_stamina,speed_walking,speed_running,speed_mounted,speed_transporting,capture_multi,male_percentage
1,Lamball,Normal,NaN,False,0,0,0,0,1,0,...,70,70,1000,100,40,400,550,160.0,1.5,50
2,Cattiva,Normal,NaN,False,0,0,0,0,1,1,...,70,70,1000,100,60,400,550,160.0,1.5,50
3,Chikipi,Normal,NaN,False,0,0,0,0,0,1,...,60,60,1000,100,50,375,550,NaN,1.5,50
4,Lifmunk,Leaf,NaN,False,0,0,1,0,1,1,...,70,70,1010,100,40,400,550,NaN,0.9,50
5,Foxparks,Fire,NaN,False,1,0,0,0,0,0,...,75,70,1040,100,80,400,550,NaN,1.1,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,Paladius,Normal,NaN,False,0,0,0,0,0,0,...,120,145,8810,100,100,800,1400,NaN,1.0,50
109,Necromus,Dark,NaN,True,0,0,0,0,0,0,...,145,120,8930,100,350,900,1600,NaN,1.0,50
110,Frostallion,Ice,NaN,False,0,0,0,0,0,0,...,140,120,8440,300,120,1000,1500,NaN,1.0,50
110B,Frostallion Noct,Dark,NaN,True,0,0,0,0,0,4,...,140,135,8560,300,120,1000,1500,NaN,1.0,50


# Read zh_hant pal names from csv

In [10]:
HANT_SPREADSHEET_ID = "1LVGMKofo6gaCPoernZ9SEIkEg5QHuayN"
HANT_SPREADSHEET_URL = f"https://docs.google.com/spreadsheets/d/{HANT_SPREADSHEET_ID}/gviz/tq?tqx=out:csv&sheet=0"
HANT_SPREADSHEET_URL

'https://docs.google.com/spreadsheets/d/1LVGMKofo6gaCPoernZ9SEIkEg5QHuayN/gviz/tq?tqx=out:csv&sheet=0'

In [11]:
# pal_zh_all_comb_df = pd.read_csv(HANT_SPREADSHEET_URL, index_col=0)
pal_zh_all_comb_df = pd.read_csv(
    "1LVGMKofo6gaCPoernZ9SEIkEg5QHuayN.csv",
    index_col=0,
)
zh_pal_names = pal_zh_all_comb_df["Unnamed: 1"][1:].str.strip()
zh_pal_names.index = pal_dict_df.index
zh_pal_names = zh_pal_names.to_dict()
zh_pal_names

/tmp/ipykernel_11654/3513942284.py:6: FutureWarning: The behavior of obj[i:j] with a float-dtype index is deprecated. In a future version, this will be treated as positional instead of label-based. For label-based slicing, use obj.loc[i:j] instead
  zh_pal_names = pal_zh_all_comb_df["Unnamed: 1"][1:].str.strip()


{'1': '棉悠悠',
 '2': '搗蛋貓',
 '3': '皮皮雞',
 '4': '翠葉鼠',
 '5': '火絨狐',
 '6': '衝浪鴨',
 '7': '伏特喵',
 '8': '新葉猿',
 '9': '燎火鹿',
 '10': '企丸丸',
 '11': '企丸王',
 '12': '電棘鼠',
 '12B': '冰刺鼠',
 '13': '葉泥泥',
 '13B': '葉泥泥(開花)',
 '14': '玉藻狐',
 '15': '啼卡爾',
 '16': '壺小象',
 '17': '瞅什魔',
 '18': '米露菲',
 '19': '寐魔',
 '20': '草莽豬',
 '21': '露娜蒂',
 '22': '遁地鼠',
 '23': '勾魂魷',
 '24': '喵絲特',
 '24B': '冰絲特',
 '25': '魯米兒',
 '26': '獵狼',
 '27': '炸蛋鳥',
 '28': '波娜兔',
 '29': '波霸牛',
 '30': '荊棘魔仙',
 '31': '鯊小子',
 '31B': '紅小鯊',
 '32': '吊縛靈',
 '32B': '冰縛靈',
 '33': '葉胖達',
 '33B': '雷胖達',
 '34': '棉花糖',
 '35': '灌木羊',
 '36': '美露帕',
 '37': '紫霞鹿',
 '37B': '祈岳鹿',
 '38': '疾風隼',
 '39': '姬小兔',
 '40': '炎魔羊',
 '40B': '暗魔羊',
 '41': '幻悅蝶',
 '42': '熾焰牛',
 '43': '趴趴鯰',
 '44': '黑鴉隱士',
 '45': '龐克蜥',
 '45B': '熱血蜥',
 '46': '月鐮魔',
 '47': '天擒鳥',
 '48': '羽箭射手',
 '48B': '山岳射手',
 '49': '鐵拳猿',
 '50': '騎士蜂',
 '51': '女皇蜂',
 '52': '笑魘貓',
 '53': '毛撣兒',
 '54': '毛老爹',
 '55': '疾旋鼬',
 '56': '雷角馬',
 '57': '吹雪狐',
 '58': '火麒麟',
 '58B': '邪麒麟',
 '59': '嚴冬鹿',
 '60': '霹靂犬'

In [12]:
pal_dict_df["name_zh"] = zh_pal_names.values()

In [13]:
pal_dict_df_cols = pal_dict_df.columns.tolist()
pal_dict_df_cols.insert(1, pal_dict_df_cols.pop(-1))
pal_dict_df = pal_dict_df[pal_dict_df_cols]

In [14]:
pal_dict_df

,name,name_zh,type_1,type_2,is_nocturnal,work_kindling,work_watering,work_planting,work_electric,work_handiwork,...,attack_shot,attack_defence,price,speed_stamina,speed_walking,speed_running,speed_mounted,speed_transporting,capture_multi,male_percentage
1,Lamball,棉悠悠,Normal,NaN,False,0,0,0,0,1,...,70,70,1000,100,40,400,550,160.0,1.5,50
2,Cattiva,搗蛋貓,Normal,NaN,False,0,0,0,0,1,...,70,70,1000,100,60,400,550,160.0,1.5,50
3,Chikipi,皮皮雞,Normal,NaN,False,0,0,0,0,0,...,60,60,1000,100,50,375,550,NaN,1.5,50
4,Lifmunk,翠葉鼠,Leaf,NaN,False,0,0,1,0,1,...,70,70,1010,100,40,400,550,NaN,0.9,50
5,Foxparks,火絨狐,Fire,NaN,False,1,0,0,0,0,...,75,70,1040,100,80,400,550,NaN,1.1,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,Paladius,聖光騎士,Normal,NaN,False,0,0,0,0,0,...,120,145,8810,100,100,800,1400,NaN,1.0,50
109,Necromus,混沌騎士,Dark,NaN,True,0,0,0,0,0,...,145,120,8930,100,350,900,1600,NaN,1.0,50
110,Frostallion,喚冬獸,Ice,NaN,False,0,0,0,0,0,...,140,120,8440,300,120,1000,1500,NaN,1.0,50
110B,Frostallion Noct,喚夜獸,Dark,NaN,True,0,0,0,0,0,...,140,135,8560,300,120,1000,1500,NaN,1.0,50


In [15]:
pal_dict = {str(k): v for k, v in pal_dict_df.to_dict(orient="index").items()}
pal_dict

{'1': {'name': 'Lamball',
  'name_zh': '棉悠悠',
  'type_1': 'Normal',
  'type_2': nan,
  'is_nocturnal': False,
  'work_kindling': 0,
  'work_watering': 0,
  'work_planting': 0,
  'work_electric': 0,
  'work_handiwork': 1,
  'work_gathering': 0,
  'work_lumbering': 0,
  'work_mining': 0,
  'work_medicine': 0,
  'work_cooling': 0,
  'work_transporting': 1,
  'work_farming': 1,
  'food': 2,
  'breedpwr': 1470,
  'attack_hp': 70,
  'attack_melee': 70,
  'attack_shot': 70,
  'attack_defence': 70,
  'price': 1000,
  'speed_stamina': 100,
  'speed_walking': 40,
  'speed_running': 400,
  'speed_mounted': 550,
  'speed_transporting': 160.0,
  'capture_multi': 1.5,
  'male_percentage': 50},
 '2': {'name': 'Cattiva',
  'name_zh': '搗蛋貓',
  'type_1': 'Normal',
  'type_2': nan,
  'is_nocturnal': False,
  'work_kindling': 0,
  'work_watering': 0,
  'work_planting': 0,
  'work_electric': 0,
  'work_handiwork': 1,
  'work_gathering': 1,
  'work_lumbering': 0,
  'work_mining': 1,
  'work_medicine': 0,
  

# Read All combination table

In [16]:
pal_all_comb_df = pd.read_excel(
    "1YgPc11dgdBUC8jXNp01b7gI6jNHoBRQGwrY_V6lXMgQ.xlsx",
    index_col=0,
    sheet_name="All Combos",
    skiprows=1,
)
pal_all_comb_df.drop("Unnamed: 1", axis=1, inplace=True)
pal_all_comb_df.columns = pal_dict_df.index
pal_all_comb_df.index = pal_dict_df.index
pal_all_comb_df

,1,2,3,4,5,6,7,8,9,10,...,104,104B,105,106,107,108,109,110,110B,111
1,Lamball,Lamball,Mau,Vixy,Lifmunk,Foxparks,Lifmunk,Jolthog,Ribunny,Sparkit,...,Broncherry,Digtoise,Eikthyrdeer,Chillet,Foxcicle,Petallia,Foxcicle,Arsox,Arsox,Petallia
2,Lamball,Cattiva,Mau,Vixy,Lifmunk,Foxparks,Lifmunk,Pengullet,Ribunny,Flambelle,...,Digtoise,Kitsun,Eikthyrdeer,Chillet,Foxcicle,Foxcicle,Foxcicle,Arsox,Petallia,Petallia
3,Mau,Mau,Chikipi,Lamball,Vixy,Hangyu,Cremis,Jolthog,Fuack,Lifmunk,...,Reindrix,Digtoise,Caprity,Dinossom,Petallia,Arsox,Arsox,Dinossom,Chillet,Arsox
4,Vixy,Vixy,Lamball,Lifmunk,Hangyu,Depresso,Hangyu,Tocotoco,Killamari,Hoocrates,...,Digtoise,Dinossom,Dumud,Arsox,Tombat,Tombat,Tombat,Petallia,Foxcicle,Foxcicle
5,Lifmunk,Lifmunk,Vixy,Hangyu,Foxparks,Jolthog,Flambelle,Bristla,Flopie,Jolthog,...,Kitsun,Chillet,Melpaca,Foxcicle,Rayhound,Rayhound,Rayhound,Foxcicle,Tombat,Tombat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,Petallia,Foxcicle,Arsox,Tombat,Rayhound,Blazehowl,Tombat,Vanwyrm,Bushi,Blazehowl,...,Astegon,Astegon,Beakon,Cryolinx,Suzaku,Paladius,Suzaku,Cryolinx,Suzaku,Suzaku
109,Foxcicle,Foxcicle,Arsox,Tombat,Rayhound,Katress,Rayhound,Vanwyrm,Incineram,Blazehowl,...,Astegon,Cryolinx,Beakon,Cryolinx,Suzaku,Suzaku,Necromus,Cryolinx,Suzaku,Suzaku
110,Arsox,Arsox,Dinossom,Petallia,Foxcicle,Rayhound,Foxcicle,Univolt,Bushi,Rayhound,...,Helzephyr,Astegon,Menasting,Cryolinx,Suzaku,Cryolinx,Cryolinx,Frostallion,Cryolinx,Cryolinx
110B,Arsox,Petallia,Chillet,Foxcicle,Tombat,Blazehowl,Tombat,Univolt,Bushi,Rayhound,...,Helzephyr,Astegon,Beakon,Cryolinx,Suzaku,Suzaku,Suzaku,Cryolinx,Frostallion Noct,Cryolinx


In [17]:
name_idx_mapping = {v["name"]: k for k, v in pal_dict.items()}
name_idx_mapping

{'Lamball': '1',
 'Cattiva': '2',
 'Chikipi': '3',
 'Lifmunk': '4',
 'Foxparks': '5',
 'Fuack': '6',
 'Sparkit': '7',
 'Tanzee': '8',
 'Rooby': '9',
 'Pengullet': '10',
 'Penking': '11',
 'Jolthog': '12',
 'Jolthog Cryst': '12B',
 'Gumoss': '13',
 'Gumoss (Special)': '13B',
 'Vixy': '14',
 'Hoocrates': '15',
 'Teafant': '16',
 'Depresso': '17',
 'Cremis': '18',
 'Daedream': '19',
 'Rushoar': '20',
 'Nox': '21',
 'Fuddler': '22',
 'Killamari': '23',
 'Mau': '24',
 'Mau Cryst': '24B',
 'Celaray': '25',
 'Direhowl': '26',
 'Tocotoco': '27',
 'Flopie': '28',
 'Mozzarina': '29',
 'Bristla': '30',
 'Gobfin': '31',
 'Gobfin Ignis': '31B',
 'Hangyu': '32',
 'Hangyu Cryst': '32B',
 'Mossanda': '33',
 'Mossanda Lux': '33B',
 'Woolipop': '34',
 'Caprity': '35',
 'Melpaca': '36',
 'Eikthyrdeer': '37',
 'Eikthyrdeer Terra': '37B',
 'Nitewing': '38',
 'Ribunny': '39',
 'Incineram': '40',
 'Incineram Noct': '40B',
 'Cinnamoth': '41',
 'Arsox': '42',
 'Dumud': '43',
 'Cawgnito': '44',
 'Leezpunk': '45

In [18]:
pal_all_comb_df = pal_all_comb_df.apply(
    lambda row: row.apply(lambda v: name_idx_mapping[v])
)
pal_all_comb_df

,1,2,3,4,5,6,7,8,9,10,...,104,104B,105,106,107,108,109,110,110B,111
1,1,1,24,14,4,5,4,12,39,7,...,86,67,37,55,57,87,57,42,42,87
2,1,2,24,14,4,5,4,10,39,70,...,67,61,37,55,57,57,57,42,87,87
3,24,24,3,1,14,32,18,12,6,4,...,59,67,35,64,87,42,42,64,55,42
4,14,14,1,4,32,17,32,27,23,15,...,67,64,43,42,68,68,68,87,57,57
5,4,4,14,32,5,12,70,30,28,12,...,61,55,36,57,60,60,60,57,68,68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,87,57,42,68,60,84,68,71,72,84,...,98,98,73,83,102,108,102,83,102,102
109,57,57,42,68,60,75,60,71,40,84,...,98,83,73,83,102,102,109,83,102,102
110,42,42,64,87,57,60,57,56,72,60,...,97,98,99,83,102,83,83,110,83,83
110B,42,87,55,57,68,84,68,56,72,60,...,97,98,73,83,102,102,102,83,110B,83


In [19]:
child_dict = {idx: set() for idx in pal_all_comb_df.index}
for idx, row in pal_all_comb_df.iterrows():
    for k, v in row.to_dict().items():
        child_dict[v].add(tuple(sorted([idx, k])))
child_dict = {
    k: sorted(v, key=lambda cb: int(cb[0].rstrip("B"))) for k, v in child_dict.items()
}
child_dict

{'1': [('1', '2'),
  ('1', '1'),
  ('2', '24'),
  ('3', '4'),
  ('14', '24'),
  ('14', '16'),
  ('16', '24B'),
  ('16', '18'),
  ('18', '24'),
  ('24B', '3')],
 '2': [('1', '18'),
  ('1', '14'),
  ('2', '2'),
  ('3', '32'),
  ('3', '32B'),
  ('16', '4'),
  ('18', '2'),
  ('24', '24B')],
 '3': [('3', '3')],
 '4': [('1', '7'),
  ('1', '5'),
  ('1', '17'),
  ('1', '70'),
  ('1', '15'),
  ('2', '7'),
  ('2', '5'),
  ('2', '32'),
  ('2', '70'),
  ('4', '4'),
  ('10', '3'),
  ('12', '24'),
  ('12B', '16'),
  ('12', '16'),
  ('12B', '3'),
  ('12', '3'),
  ('14', '7'),
  ('14', '5'),
  ('14', '32B'),
  ('14', '32'),
  ('14', '70'),
  ('14', '4'),
  ('15', '2'),
  ('15', '24'),
  ('15', '16'),
  ('16', '17'),
  ('17', '24'),
  ('17', '3'),
  ('18', '70'),
  ('18', '7'),
  ('18', '5'),
  ('18', '32B'),
  ('18', '32'),
  ('24B', '4'),
  ('24', '5'),
  ('24B', '7'),
  ('24B', '32B'),
  ('24B', '32'),
  ('32B', '4'),
  ('32', '4')],
 '5': [('1', '6'),
  ('1', '30'),
  ('2', '27'),
  ('2', '6'),
  (

In [21]:
import json


def json_dump(data, output_file_name):
    with open(output_file_name, "w") as fout:
        json.dump(data, fout, ensure_ascii=False)

In [22]:
json_dump(child_dict, "child_dict.json")
json_dump(pal_all_comb_df.to_dict(orient="index"), "all_comb.json")
pal_dict_df.to_json("pal_dict.json", orient="index", force_ascii=False)

In [23]:
i18n_zh_hant_dict, i18n_en_dict = {"pal": {}}, {"pal": {}}
for k, v in pal_dict.items():
    i18n_en_dict["pal"][f"pal_{k.zfill(3)}"] = v["name"]
    i18n_zh_hant_dict["pal"][f"pal_{k.zfill(3)}"] = v["name_zh"]
i18n_en_dict

{'pal': {'pal_001': 'Lamball',
  'pal_002': 'Cattiva',
  'pal_003': 'Chikipi',
  'pal_004': 'Lifmunk',
  'pal_005': 'Foxparks',
  'pal_006': 'Fuack',
  'pal_007': 'Sparkit',
  'pal_008': 'Tanzee',
  'pal_009': 'Rooby',
  'pal_010': 'Pengullet',
  'pal_011': 'Penking',
  'pal_012': 'Jolthog',
  'pal_12B': 'Jolthog Cryst',
  'pal_013': 'Gumoss',
  'pal_13B': 'Gumoss (Special)',
  'pal_014': 'Vixy',
  'pal_015': 'Hoocrates',
  'pal_016': 'Teafant',
  'pal_017': 'Depresso',
  'pal_018': 'Cremis',
  'pal_019': 'Daedream',
  'pal_020': 'Rushoar',
  'pal_021': 'Nox',
  'pal_022': 'Fuddler',
  'pal_023': 'Killamari',
  'pal_024': 'Mau',
  'pal_24B': 'Mau Cryst',
  'pal_025': 'Celaray',
  'pal_026': 'Direhowl',
  'pal_027': 'Tocotoco',
  'pal_028': 'Flopie',
  'pal_029': 'Mozzarina',
  'pal_030': 'Bristla',
  'pal_031': 'Gobfin',
  'pal_31B': 'Gobfin Ignis',
  'pal_032': 'Hangyu',
  'pal_32B': 'Hangyu Cryst',
  'pal_033': 'Mossanda',
  'pal_33B': 'Mossanda Lux',
  'pal_034': 'Woolipop',
  'pal_